In [331]:
import numpy as np #import numpy library
import pandas as pd #import pandas library
import matplotlib.pyplot as plt #import matplot library
%matplotlib inline

import glob
import os

import nltk.classify.util
from astropy.table import Table, Column

import string

from sklearn import svm
from sklearn import metrics

In [332]:
 def loadMetricData(user, directory):
    path =  user + "/" + directory + "/"
#     print(path)
    all_files = glob.glob(os.path.join(path, "*.csv")) 
#     print(all_files)
    data = pd.DataFrame()
    list_ = []
    for file_ in all_files:
        data = pd.read_csv(file_,index_col=None, header=0)
        list_.append(data)
        data = pd.concat(list_)
    return data

In [333]:
# # clean times function
def cleanTime(df, col):
    df.col = pd.to_datetime(df.col)

In [334]:
metrics = loadMetricData('sarahYee', "metrics")
metrics.tail(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp
1435,2016-05-04 23:55:00,71,0,0.9,0.000067,93.2,93.2
1436,2016-05-04 23:56:00,69,0,1.0,0.000067,94.1,94.1
1437,2016-05-04 23:57:00,68,0,0.9,0.000069,94.1,94.1
1438,2016-05-04 23:58:00,66,0,1.0,0.000071,94.1,94.1
1439,2016-05-04 23:59:00,65,0,0.9,0.000067,94.1,94.1


In [335]:
metrics.dtypes

timestamp     object
heartrate    float64
steps        float64
calories     float64
gsr          float64
skintemp     float64
airtemp      float64
dtype: object

In [336]:
# clean metric data

metrics.timestamp = pd.to_datetime(metrics.timestamp)

metrics.heartrate = metrics.heartrate.fillna(0)
metrics.steps = metrics.steps.fillna(0)
metrics.calories = metrics.calories.fillna(0)
metrics.gsr = metrics.gsr.fillna(0)
metrics.skintemp = metrics.skintemp.fillna(0)
metrics.airtemp = metrics.airtemp.fillna(0)

# metrics.head(5)

In [337]:
metrics = metrics[metrics.heartrate != 0]
metrics.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp
705,2016-04-22 11:45:00,83,478,1167.40002,0.000000,0.0,0.0
706,2016-04-22 11:46:00,80,0,1.60000,0.000054,89.6,88.7
708,2016-04-22 11:48:00,120,0,3.00000,0.000000,0.0,0.0
709,2016-04-22 11:49:00,82,0,1.10000,0.000000,0.0,0.0
710,2016-04-22 11:50:00,81,0,1.30000,0.000064,88.7,87.8


In [338]:
activities = loadMetricData('sarahYee', "activities")
# activities.tail(5)

In [339]:
sleep = loadMetricData('sarahYee', "sleep")
# sleep.tail(5)

In [340]:
# sleep.dtypes

In [341]:
journal = loadMetricData('sarahYee', "journal")
# journal.tail(5)

In [342]:
# journal.dtypes

In [343]:
# clean metric data
journal.starttime= pd.to_datetime(journal.starttime)
journal.endtime= pd.to_datetime(journal.endtime)

journal.qualitative = journal.qualitative.fillna(0)
journal.excited = journal.excited.fillna(0)
journal.happy = journal.happy.fillna(0)
journal.calm = journal.calm.fillna(0)
journal.anxious = journal.anxious.fillna(0)
journal.sad = journal.sad.fillna(0)
journal.angry = journal.angry.fillna(0)
journal.hungry = journal.hungry.fillna(0)
journal.tired = journal.tired.fillna(0)
journal.bored = journal.bored.fillna(0)
journal.ansaan = journal.ansaan.fillna(0)
journal.exhaca = journal.exhaca.fillna(0)

print(journal.shape)
journal.head(5)
# journal.tail(5)

(127, 24)


,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,...,angry,hungry,tired,bored,exhaca,ansaan,exha,haca,anan,ansa
0,2016-04-22 13:48:00,2016-04-22 16:12:00,4/22/16,1:48:00 PM,4:12:00 PM,writing thesis,barb's office,anxious,6,0,...,0,0,0,0,0,6,0,0,6,6
1,2016-04-22 16:12:00,2016-04-22 17:15:00,4/22/16,4:12:00 PM,5:15:00 PM,writing thesis,barb's office,anxious,6,0,...,0,0,0,0,0,6,0,0,6,6
2,2016-04-22 17:15:00,2016-04-22 19:05:00,4/22/16,5:15:00 PM,7:05:00 PM,eating russian dumplings with annie then watch...,5th floor lounge,happy,10,0,...,0,0,0,0,10,0,10,10,0,0
3,2016-04-22 19:05:00,2016-04-22 19:15:00,4/22/16,7:05:00 PM,7:15:00 PM,photofeast,3rd floor,excited/anxious,5/3,5,...,0,0,0,0,5,3,5,0,3,3
4,2016-04-22 19:15:00,2016-04-22 19:25:00,4/22/16,7:15:00 PM,7:25:00 PM,naked picture of charles in photofeast,3rd floor,what can only be described as traumatic shock,10,0,...,0,0,0,0,0,0,0,0,0,0


In [344]:

journal['boolexcited'] =  journal['excited'] > 0
journal['boolhappy'] =  journal['happy'] > 0
journal['boolcalm'] =  journal['calm'] > 0
journal['boolanxious'] =  journal['anxious'] > 0
journal['boolsad'] =  journal['sad'] > 0
journal['boolangry'] =  journal['angry'] > 0
journal['booltired'] =  journal['tired'] > 0
journal['boolhungry'] =  journal['hungry'] > 0
journal['boolbored'] =  journal['bored'] > 0
journal['exhaca'] =  journal['exhaca'] > 0
journal['ansaan'] =  journal['ansaan'] > 0

journal['exha'] =  journal['exha'] > 0
journal['haca'] =  journal['haca'] > 0
journal['anan'] =  journal['anan'] > 0
journal['ansa'] =  journal['anan'] > 0

    
boolexcited = journal.iloc[:, 18].values.reshape(journal.shape[0], 1)
boolhappy = journal.iloc[:, 19].values.reshape(journal.shape[0], 1)
boolcalm = journal.iloc[:, 20].values.reshape(journal.shape[0], 1)
boolanxious = journal.iloc[:, 21].values.reshape(journal.shape[0], 1)
boolsad = journal.iloc[:, 22].values.reshape(journal.shape[0], 1)
boolangry = journal.iloc[:, 23].values.reshape(journal.shape[0], 1)
booltired = journal.iloc[:, 24].values.reshape(journal.shape[0], 1)
boolhungry = journal.iloc[:, 25].values.reshape(journal.shape[0], 1)
boolbored = journal.iloc[:, 26].values.reshape(journal.shape[0], 1)
exhaca = journal.iloc[:, 27].values.reshape(journal.shape[0], 1)
ansaan = journal.iloc[:, 28].values.reshape(journal.shape[0], 1)
exha = journal.iloc[:, 29].values.reshape(journal.shape[0], 1)
haca = journal.iloc[:, 30].values.reshape(journal.shape[0], 1)
anan = journal.iloc[:, 31].values.reshape(journal.shape[0], 1)
ansa = journal.iloc[:, 32].values.reshape(journal.shape[0], 1)

journal.head(5)


,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,...,ansa,boolexcited,boolhappy,boolcalm,boolanxious,boolsad,boolangry,booltired,boolhungry,boolbored
0,2016-04-22 13:48:00,2016-04-22 16:12:00,4/22/16,1:48:00 PM,4:12:00 PM,writing thesis,barb's office,anxious,6,0,...,True,False,False,False,True,False,False,False,False,False
1,2016-04-22 16:12:00,2016-04-22 17:15:00,4/22/16,4:12:00 PM,5:15:00 PM,writing thesis,barb's office,anxious,6,0,...,True,False,False,False,True,False,False,False,False,False
2,2016-04-22 17:15:00,2016-04-22 19:05:00,4/22/16,5:15:00 PM,7:05:00 PM,eating russian dumplings with annie then watch...,5th floor lounge,happy,10,0,...,False,False,True,False,False,False,False,False,False,False
3,2016-04-22 19:05:00,2016-04-22 19:15:00,4/22/16,7:05:00 PM,7:15:00 PM,photofeast,3rd floor,excited/anxious,5/3,5,...,True,True,False,False,True,False,False,False,False,False
4,2016-04-22 19:15:00,2016-04-22 19:25:00,4/22/16,7:15:00 PM,7:25:00 PM,naked picture of charles in photofeast,3rd floor,what can only be described as traumatic shock,10,0,...,False,False,False,False,False,False,False,False,False,False


In [345]:
# for i in range(0, sarahJournal.shape[0]):
#     sarahJournal['ex'] = (0< sarahJournal.iloc[i, 18].astype(int))
#     sarahJournal['ha'] = sarahJournal.iloc[i, 19]== True
#     sarahJournal['ca'] = sarahJournal.iloc[i, 20]== True
#     sarahJournal['ys'] = sarahJournal['ex'].astype(int) + sarahJournal['ha'].astype(int) + sarahJournal['ca'].astype(int)

# print(sarahJournal['ex'])
    

In [346]:
# def makeEmoIndicators(newvar, journalData, journal_emo1_loc, journal_emo2_loc, journal_emo3_loc):
  
#     journalData[newvar] = False
    
#     for i in range(0, journalData.shape[0]):
#         journalData['emo1'] = (journalData.iloc[i, journal_emo1_loc]) == True)
#         journalData['emo2'] = (journalData.iloc[i, journal_emo2_loc] == True)
#         journalData['emo3'] = (journalData.iloc[i, journal_emo3_loc] == True)
#         journalData['ysum'] = journalData['emo1'].astype(int) + journalData['emo2'].astype(int) + journalData['emo3'].astype(int)
# #         print(journalData['emo1'].astype(int) + journalData['emo2'].astype(int) + journalData['emo3'].astype(int))
#         journalData.loc[journalData['ysum']!=0, newvar] = True
        
#     print(journalData['emo1'].values)
#     journalData = journalData.drop(['emo1', 'emo2', 'emo3', 'ysum'], axis=1)
#     return journalData

In [347]:
# sarahJournal.dtypes

In [348]:
# testData= makeEmoIndicators('exhaca', sarahJournal, 18, 19, 20)
# # print(testData.describe())

In [349]:
# sarahJournal.boolexhapcal.convert_objects(convert_numeric=True)

In [350]:
# function that takes any journal or activity data and creates indicators for activities or emotions

def makeIndicators(newvar, metricData, journalData, journal_startdatetime_loc, journal_enddatetime_loc, journal_activity_loc):
    '''
    Create boolean indicator variable for activities or emotions
    newvar, string for name of new indicator variable
    activity, string that matches the name of the activity or emotion in the journalData data frame
    bioData, data frame of input biometric data
    journalData, data frame of input journal or activity data
    journal_startdatetime_loc, int of position of the start datetime in journalData
    journal_enddatetime_loc, int of position of the end datetime in journalData
    journal_activity_loc, int of position of the activity or emotion in journalData
    '''

    metricData[newvar] = False

    for i in range(0, journalData.shape[0]):
        metricData['xlt'] = (metricData['timestamp'] >= pd.to_datetime(journalData.iloc[i, journal_startdatetime_loc]))
        metricData['xgt'] = (metricData['timestamp'] <= pd.to_datetime(journalData.iloc[i, journal_enddatetime_loc]))
#         metricData['xw'] = (journalData.iloc[i, journal_activity_loc])
        metricData['xw'] = (journalData.iloc[i, journal_activity_loc] == True)
        metricData['xsum'] = metricData['xlt'].astype(int) + metricData['xgt'].astype(int) + metricData['xw'].astype(int)
        metricData.loc[metricData['xsum']==3, newvar] = True
    
    metricData = metricData.drop(['xlt', 'xgt', 'xw', 'xsum'], axis=1)
    return metricData

In [351]:
# indicator for walking
totalData = makeIndicators('excitement', metrics, journal, 0, 1, 18)
totalData = makeIndicators('happy', metrics, journal, 0, 1, 19)
totalData = makeIndicators('calm', metrics, journal, 0, 1, 20)
totalData = makeIndicators('anxious', metrics, journal, 0, 1, 21)
totalData = makeIndicators('sad', metrics, journal, 0, 1, 22)
totalData = makeIndicators('angry', metrics, journal, 0, 1, 23)
totalData = makeIndicators('hungry', metrics, journal, 0, 1, 24)
totalData = makeIndicators('tired', metrics, journal, 0, 1, 25)
totalData = makeIndicators('bored', metrics, journal, 0, 1, 26)

totalData = makeIndicators('exhaca', metrics, journal, 0, 1, 27)
totalData = makeIndicators('ansaan', metrics, journal, 0, 1, 28)

totalData = makeIndicators('exha', metrics, journal, 0, 1, 29)
totalData = makeIndicators('haca', metrics, journal, 0, 1, 30)
totalData = makeIndicators('anan', metrics, journal, 0, 1, 31)
totalData = makeIndicators('ansa', metrics, journal, 0, 1, 32)




In [352]:
print(totalData.describe())

          heartrate         steps      calories           gsr      skintemp  \
count  17279.000000  17279.000000  17279.000000  17279.000000  17279.000000   
mean      87.660281      8.785231      1.822420      0.045547     88.762446   
std       18.073892     26.172463      8.982735      0.417669      6.415743   
min       57.000000      0.000000      0.000000      0.000000      0.000000   
25%       71.000000      0.000000      1.000000      0.000054     86.900000   
50%       87.000000      0.000000      1.200000      0.000062     89.600000   
75%       99.000000      0.000000      1.800000      0.000081     91.400000   
max      168.000000    695.000000   1167.400020     10.500000     97.700000   

            airtemp excitement     happy       calm   anxious    ...      \
count  17279.000000      17279     17279      17279     17279    ...       
mean      85.298235   0.184038  0.245327  0.0478037   0.17773    ...       
std        7.025071   0.387527  0.430293   0.213357  0.38229

In [353]:
print(totalData.head())

              timestamp  heartrate  steps    calories       gsr  skintemp  \
705 2016-04-22 11:45:00         83    478  1167.40002  0.000000       0.0   
706 2016-04-22 11:46:00         80      0     1.60000  0.000054      89.6   
708 2016-04-22 11:48:00        120      0     3.00000  0.000000       0.0   
709 2016-04-22 11:49:00         82      0     1.10000  0.000000       0.0   
710 2016-04-22 11:50:00         81      0     1.30000  0.000064      88.7   

     airtemp excitement  happy   calm  ...    angry hungry  tired  bored  \
705      0.0      False  False  False  ...    False  False  False  False   
706     88.7      False  False  False  ...    False  False  False  False   
708      0.0      False  False  False  ...    False  False  False  False   
709      0.0      False  False  False  ...    False  False  False  False   
710     87.8      False  False  False  ...    False  False  False  False   

    exhaca ansaan   exha   haca   anan   ansa  
705  False  False  False  False 

In [354]:
totalData.to_csv('sarahTotal.csv')

In [355]:
totalData.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp,excitement,happy,calm,...,angry,hungry,tired,bored,exhaca,ansaan,exha,haca,anan,ansa
705,2016-04-22 11:45:00,83,478,1167.40002,0.000000,0.0,0.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
706,2016-04-22 11:46:00,80,0,1.60000,0.000054,89.6,88.7,False,False,False,...,False,False,False,False,False,False,False,False,False,False
708,2016-04-22 11:48:00,120,0,3.00000,0.000000,0.0,0.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
709,2016-04-22 11:49:00,82,0,1.10000,0.000000,0.0,0.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
710,2016-04-22 11:50:00,81,0,1.30000,0.000064,88.7,87.8,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [356]:
# find day of week (0-6, Mon-Sun)
totalData['weekday'] = totalData['timestamp'].dt.weekday
totalData['month'] = totalData['timestamp'].dt.month
totalData['hour'] = totalData['timestamp'].dt.hour
totalData['min'] = totalData['timestamp'].dt.minute


print(totalData.shape)
print(totalData.head(5))

(17279, 26)
              timestamp  heartrate  steps    calories       gsr  skintemp  \
705 2016-04-22 11:45:00         83    478  1167.40002  0.000000       0.0   
706 2016-04-22 11:46:00         80      0     1.60000  0.000054      89.6   
708 2016-04-22 11:48:00        120      0     3.00000  0.000000       0.0   
709 2016-04-22 11:49:00         82      0     1.10000  0.000000       0.0   
710 2016-04-22 11:50:00         81      0     1.30000  0.000064      88.7   

     airtemp excitement  happy   calm ... exhaca ansaan   exha   haca   anan  \
705      0.0      False  False  False ...  False  False  False  False  False   
706     88.7      False  False  False ...  False  False  False  False  False   
708      0.0      False  False  False ...  False  False  False  False  False   
709      0.0      False  False  False ...  False  False  False  False  False   
710     87.8      False  False  False ...  False  False  False  False  False   

      ansa weekday month hour min  
705  Fal

In [357]:
heartrate = totalData.iloc[:, 1].values.reshape(totalData.shape[0], 1)
steps = totalData.iloc[:, 2].values.reshape(totalData.shape[0], 1)
calories = totalData.iloc[:, 3].values.reshape(totalData.shape[0], 1)
gsr = totalData.iloc[:, 4].values.reshape(totalData.shape[0], 1)
skintemp = totalData.iloc[:, 5].values.reshape(totalData.shape[0], 1)
airtemp = totalData.iloc[:, 6].values.reshape(totalData.shape[0], 1)
# activity = totalData.iloc[:, 17].values.reshape(totalData.shape[0], 1)
# exercise = totalData.iloc[:, 17].values.reshape(totalData.shape[0], 1)
# thesis = totalData.iloc[:, 18].values.reshape(totalData.shape[0], 1)
# sally = totalData.iloc[:, 19].values.reshape(totalData.shape[0], 1)
# barb = totalData.iloc[:, 20].values.reshape(totalData.shape[0], 1)
# sol = totalData.iloc[:, 21].values.reshape(totalData.shape[0], 1)
# arash = totalData.iloc[:, 22].values.reshape(totalData.shape[0], 1)
# work = totalData.iloc[:, 23].values.reshape(totalData.shape[0], 1)
# home = totalData.iloc[:, 24].values.reshape(totalData.shape[0], 1)
# DayOfWeek = totalData.iloc[:, 25].values.reshape(totalData.shape[0], 1)
# Month = totalData.iloc[:, 26].values.reshape(totalData.shape[0], 1)
weekday = totalData.iloc[:, 7].values.reshape(totalData.shape[0], 1)
# month = totalData.iloc[:, 29].values.reshape(totalData.shape[0], 1)
hour = totalData.iloc[:, 8].values.reshape(totalData.shape[0], 1)
curmin = totalData.iloc[:, 9].values.reshape(totalData.shape[0], 1)

In [358]:
# concatenate to numpy dataset
X = np.concatenate((
        heartrate, 
        steps, 
        calories, 
        gsr, 
        skintemp, 
        airtemp, 
#         exercise, 
#         thesis, 
#         sally, 
#         barb, 
#         sol, 
#         arash, 
#         work, 
#         home, 
        weekday, 
        hour, 
        curmin
    ), axis=1)

X.shape

(17279, 9)

# Positive ( Excited, Happy, Calm)

In [359]:
# define y


y = totalData.exhaca.values

# y = totalData.excitement.values
y.shape

(17279,)

In [360]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [361]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [362]:
# report on training and test sets
global SVMerror, SVMacc, SVMtp, SVMtn, LRerror, LRacc, LRtp, LRtn, NBerror, NBacc, NBtp, NBtn, Perror, Pacc, Ptp, Ptn



def print_results(model):
    #print('Error rate on training set: ')
    erTRAIN = ((y_train != y_pred).sum() / X_train.shape[0])
    #print('Accuracy rate on training set: ')
    AccTRAIN = (1 - (y_train != y_pred).sum() / X_train.shape[0])
    #print('True positive rate on training tet:')
    TruPosTRAIN = ((y_train==True) & (y_pred==True)).sum() / y_train.sum()
    #TruNegTEST = (((y_train==False) & (y_pred_train==False)).sum() / (y_train.shape[0] - y_train.sum()))
    #print('**************')
    #('Error rate on test set: ')
    erTEST = ((y_test != y_pred_test).sum() / X_test.shape[0])
    #print('Accuracy rate on test set: ')
    AccTEST = (1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    #print('True positive rate on test set')
    TruPosTEST = (((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    #print('True negative rate on test set')
    TruNegTEST = (((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))
    data_rows = [('Error Rate', erTRAIN, erTEST),
                 ('Accuracy Rate', AccTRAIN, AccTEST),
                 ('True Positives', TruPosTRAIN, TruPosTEST),
                 ('True Negatives', '--', TruNegTEST)]
    t = Table(rows=data_rows, names=(model, 'Training Set', 'Test Set'), meta={'name': model + ': Training and Test Set Results'})
    print(t)
    if model == 'SVM':
        SVMerror = erTEST
        SVMacc = AccTEST
        SVMtp = TruPosTEST
        SVMtn = TruNegTEST
        return(SVMerror, SVMacc, SVMtp, SVMtn)
    elif model == 'Logistic Regression':
        LRerror = erTEST
        LRacc = AccTEST
        LRtp = TruPosTEST
        LRtn = TruNegTEST
        return(LRerror, LRacc, LRtp, LRtn)
    elif model == 'Naive Bayes':
        NBerror = erTEST
        NBacc = AccTEST
        NBtp = TruPosTEST
        NBtn = TruNegTEST
        return(NBerror, NBacc, NBtp, NBtn)
    elif model == 'Perceptron':
        Perror = erTEST
        Pacc = AccTEST
        Ptp = TruPosTEST
        Ptn = TruNegTEST
        return(Perror, Pacc, Ptp, Ptn)
    print('done')
      
    #t.show_in_browser(jsviewer=True) 
    
    
def all_models_table():
    all_rows = [('SVM', SVMerror, SVMacc, SVMtp, SVMtn),
            ('Logistic Regression', LRerror, LRacc, LRtp, LRtn),
            ('Naive Bayes', NBerror, NBacc, NBtp, NBtn),
            ('Perceptron', Perror, Pacc, Ptp, Ptn)]
    tt = Table(rows=all_rows, names=('', 'Error Rate', 'Accuracy', 'True +', 'True -'), meta={'3/15/2016'})
    print(tt)

## Positive ( Excited, Happy, Calm) Model Results

In [363]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0117403885903 0.0117669753086
 Accuracy Rate   0.98825961141  0.988233024691
True Positives  0.988130563798   0.98465266559
True Negatives              --  0.989356310188


(0.011766975308641976,
 0.98823302469135799,
 0.98465266558966069,
 0.98935631018753167)

In [364]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0138073584126 0.0111882716049
      Accuracy Rate  0.986192641587  0.988811728395
     True Positives  0.995919881306  0.998384491115
     True Negatives              --  0.985808413583


(0.011188271604938271,
 0.98881172839506171,
 0.99838449111470118,
 0.98580841358337556)

In [365]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0367920628359 0.0341435185185
 Accuracy Rate  0.963207937164  0.965856481481
True Positives             1.0             1.0
True Negatives              --  0.955144450076


(0.034143518518518517, 0.96585648148148151, 1.0, 0.95514445007602633)

In [366]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0195948739148 0.0165895061728
 Accuracy Rate  0.980405126085  0.983410493827
True Positives             1.0             1.0
True Negatives              --  0.978205778003


(0.016589506172839507, 0.9834104938271605, 1.0, 0.97820577800304109)

# Negative ( Anxious, Sad, Angry)

In [367]:
# define y
y = totalData.ansaan.values
y.shape

(17279,)

In [368]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [369]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Negative ( Anxious, Sad, Angry) Model Results

In [370]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0442331541959 0.0437885802469
 Accuracy Rate  0.955766845804  0.956211419753
True Positives             0.0             0.0
True Negatives              --  0.999193710945


(0.04378858024691358, 0.95621141975308643, 0.0, 0.9991937109453739)

In [371]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0440677966102 0.0422453703704
      Accuracy Rate   0.95593220339   0.95775462963
     True Positives  0.574144486692  0.556053811659
     True Negatives              --  0.975811328361


(0.042245370370370371,
 0.95775462962962965,
 0.55605381165919288,
 0.97581132836121753)

In [372]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0455560148822 0.0459104938272
 Accuracy Rate  0.954443985118  0.954089506173
True Positives  0.121673003802  0.116591928251
True Negatives              --   0.99173553719


(0.045910493827160496,
 0.9540895061728395,
 0.11659192825112108,
 0.99173553719008267)

In [373]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.167920628359 0.182291666667
 Accuracy Rate 0.832079371641 0.817708333333
True Positives 0.979087452471 0.964125560538
True Negatives             -- 0.811126788954


(0.18229166666666666,
 0.81770833333333337,
 0.9641255605381166,
 0.81112678895383994)

# Happy/Calm 

In [374]:
# define y
y = totalData.haca.values
y.shape

(17279,)

In [375]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [376]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Happy/Calm  Model Results

In [377]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0844150475403 0.0800540123457
 Accuracy Rate   0.91558495246  0.919945987654
True Positives 0.0449859418932 0.0460829493088
True Negatives              --  0.999789473684


(0.080054012345679007,
 0.91994598765432101,
 0.046082949308755762,
 0.99978947368421056)

In [378]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0706076891277 0.0655864197531
      Accuracy Rate  0.929392310872  0.934413580247
     True Positives  0.427366447985  0.447004608295
     True Negatives              --  0.978947368421


(0.06558641975308642,
 0.93441358024691357,
 0.44700460829493088,
 0.97894736842105268)

In [379]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0950806118231 0.0960648148148
 Accuracy Rate  0.904919388177  0.903935185185
True Positives  0.419868791003  0.428571428571
True Negatives              --  0.947368421053


(0.096064814814814811,
 0.90393518518518523,
 0.42857142857142855,
 0.94736842105263153)

In [380]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0851591566763 0.0804398148148
 Accuracy Rate  0.914840843324  0.919560185185
True Positives 0.0421743205248 0.0483870967742
True Negatives              --  0.999157894737


(0.080439814814814811,
 0.91956018518518523,
 0.048387096774193547,
 0.99915789473684213)

# Anxious/Angry

In [381]:
# define y
y = totalData.anan.values
y.shape

(17279,)

In [382]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [383]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Anxious/Angry Model Results

In [384]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0101694915254 0.0119598765432
 Accuracy Rate  0.989830508475  0.988040123457
True Positives             0.0             0.0
True Negatives              --  0.999414634146


(0.011959876543209876, 0.98804012345679015, 0.0, 0.99941463414634146)

In [385]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0095907399752 0.0113811728395
      Accuracy Rate  0.990409260025   0.98861882716
     True Positives             0.0             0.0
     True Negatives              --             1.0


(0.011381172839506173, 0.98861882716049387, 0.0, 1.0)

In [386]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0455560148822 0.0515046296296
 Accuracy Rate  0.954443985118   0.94849537037
True Positives  0.417391304348  0.406779661017
True Negatives              --  0.954731707317


(0.051504629629629629,
 0.94849537037037035,
 0.40677966101694918,
 0.95473170731707313)

In [387]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set       Test Set   
-------------- ---------------- ---------------
    Error Rate 0.00950806118231 0.0113811728395
 Accuracy Rate   0.990491938818   0.98861882716
True Positives              0.0             0.0
True Negatives               --             1.0


(0.011381172839506173, 0.98861882716049387, 0.0, 1.0)

# Excited/Happy

In [388]:
# define y
y = totalData.exha.values
y.shape

(17279,)

In [389]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [390]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Excited/Happy Model Values

In [391]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set     Test Set   
-------------- --------------- --------------
    Error Rate 0.0908639933857        0.09375
 Accuracy Rate  0.909136006614        0.90625
True Positives  0.833869239014 0.866995073892
True Negatives              --  0.90958560067


(0.09375, 0.90625, 0.86699507389162567, 0.90958560066973626)

In [392]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0725919801571 0.0763888888889
      Accuracy Rate  0.927408019843  0.923611111111
     True Positives 0.0878885316184 0.0738916256158
     True Negatives              --  0.995814148179


(0.076388888888888895,
 0.92361111111111116,
 0.073891625615763554,
 0.99581414817915448)

In [393]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0807771806532 0.0860339506173
 Accuracy Rate  0.919222819347  0.913966049383
True Positives  0.167202572347  0.162561576355
True Negatives              --   0.97781498535


(0.086033950617283944,
 0.91396604938271608,
 0.1625615763546798,
 0.97781498534951861)

In [394]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set    
-------------- --------------- ----------------
    Error Rate 0.0769739561802  0.0790895061728
 Accuracy Rate   0.92302604382   0.920910493827
True Positives 0.0096463022508 0.00492610837438
True Negatives              --   0.998744244454


(0.07908950617283951,
 0.9209104938271605,
 0.0049261083743842365,
 0.99874424445374632)

# Anxious/Sad

In [395]:
# define y
y = totalData.ansa.values
y.shape

(17279,)

In [396]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [397]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Anxious/Sad Model Results

In [398]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set       Test Set   
-------------- ---------------- ---------------
    Error Rate  0.0576271186441 0.0569058641975
 Accuracy Rate   0.942372881356  0.943094135802
True Positives 0.00144717800289             0.0
True Negatives               --  0.998978340826


(0.056905864197530867, 0.94309413580246915, 0.0, 0.99897834082550063)

In [399]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0568830095081 0.0549768518519
      Accuracy Rate  0.943116990492  0.945023148148
     True Positives  0.013024602026 0.0172413793103
     True Negatives              --             1.0


(0.054976851851851853, 0.94502314814814814, 0.017241379310344827, 1.0)

In [400]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0424968995453 0.0491898148148
 Accuracy Rate  0.957503100455  0.950810185185
True Positives  0.552821997106  0.489655172414
True Negatives              --  0.978136493666


(0.049189814814814818,
 0.95081018518518523,
 0.48965517241379308,
 0.97813649366571309)

In [401]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0863166597768 0.0933641975309
 Accuracy Rate  0.913683340223  0.906635802469
True Positives  0.781476121563  0.762068965517
True Negatives              --  0.915202288517


(0.093364197530864196,
 0.90663580246913578,
 0.76206896551724135,
 0.91520228851655083)